In [66]:
from time import sleep
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains

from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
from string import punctuation
import re
import nltk
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

In [67]:
driver = webdriver.Chrome('C:/Users/bandi/Desktop/Text Analytics/TA Session/chromedriver_win32/chromedriver')
driver.get('https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p702')

In [3]:
#Creating empty data frame to store user_id, dates and comments from ~5K users.
comments = pd.DataFrame(columns = ['Date','user_id','comments'])

j = 702
while (j>=1):
    # Running while loop only till we get 5K comments 
    if (len(comments)<5000):
        url = 'https://forums.edmunds.com/discussion/2864/general/x/entry-level-luxury-performance-sedans/p' + str(j)
        driver.get(url)
        ids = driver.find_elements_by_xpath("//*[contains(@id,'Comment_')]")
        comment_ids = []
        for i in ids:
            comment_ids.append(i.get_attribute('id'))

        for x in comment_ids:
            #Extract dates from for each user on a page
            user_date = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[2]/span[1]/a/time')[0]
            date = user_date.get_attribute('title')

            #Extract user ids from each user on a page
            user_title = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[2]/div[1]/span[1]/a[1]')[0]
            userid = user_title.get_attribute('title') 

            #Extract Message for each user on a page
            user_message = driver.find_elements_by_xpath('//*[@id="' + x +'"]/div/div[3]')[0]
            comment = user_message.text
           #Adding date, userid and comment for each user in a dataframe    
            comments.loc[len(comments)] = [date,userid,comment]
        j=j-1
    else:
        break
        

In [ ]:
import copy
comments_copy = copy.deepcopy(comments)

In [30]:
def remove_space(s):
    return s.replace("\n"," ")
comments_copy['comments'] = comments_copy['comments'].apply(remove_space)

In [31]:
comments_copy.to_csv('comments.csv', header=True, sep=',') 

In [40]:
def removepunc(item):
    for p in punctuation:
        item = item.replace(p,'')
    return item

def remove_numbers(s):
    return re.sub("\S*\d\S*", "", s).strip()

def lowerize(x):
    return x.lower()

comments_copy['comments_clean'] = comments_copy['comments'].apply(removepunc).apply(lowerize)

In [41]:
comments_copy['comments_clean'][2]

'circlew saidthe lease rate is the factor that stops me cold from taking the leap since i dont track my cars it would do fine for my needs but i would assume an m2 would surpass the stinger on the track compared to the giuliayes the completely noncomparable m2 would beat the stinger on track as would a corvette or sportbike btw17 f150 crew 27 67 coronet rt 14 towncountry limited 09 lr2 hse 44car history and counting'

In [112]:
models = pd.read_csv("models.csv", header = None, names = ['brand','model'], encoding='windows-1252')

models['brand'] = models['brand'].apply(removepunc)

def model_to_brand(s):
    for i in models.index.values:
        s = s.replace(models["model"][i].lower(),models["brand"][i].lower())
    return s
comments_copy['comments_model_replace'] = comments_copy['comments_clean'].apply(model_to_brand)

In [113]:
temp = copy.deepcopy(comments_copy)

In [114]:
import nltk
from nltk import word_tokenize

In [115]:
temp['comments_appear'] = temp['comments_model_replace'].apply(word_tokenize).apply(set).apply(list)

# Remove stop words
stop_words = set(stopwords.words('english'))
def remove_stopwords(s):
    return [w for w in s if not w in stop_words] 
    
temp['final_comments'] =  temp['comments_appear'].apply(remove_stopwords)
#filtered_sentence = [w for w in temp['comments_appear'] if not w in stop_words]

In [116]:
count = []
for i in range(len(temp)):
    count+=temp['final_comments'][i]

In [117]:
from nltk import FreqDist
word_freq = nltk.FreqDist(count)

In [119]:
models_unique = models['brand'].drop_duplicates().tolist()
models_unique

['acura',
 'audi',
 'bmw',
 'buick',
 'cadillac',
 'car',
 'chevrolet',
 'chrysler',
 'dodge',
 'ford',
 'honda',
 'hyndai kia',
 'hyundai',
 'infiniti',
 'kia',
 'lincoln',
 'mazda',
 'mercedes',
 'mercedes benz',
 'mercedesbenz',
 'mercury',
 'mitsubishi',
 'nissan',
 'pontiac',
 'problem',
 'saturn',
 'seat',
 'sedan',
 'subaru',
 'suzuki',
 'toyata',
 'toyota',
 'volkswagen',
 'volkwagen',
 'volvo']

In [126]:
top_words = word_freq.most_common(500)
top_brands = []
for (key, items) in top_words:
    if key in models_unique:
        model_counts = (key,items)
        top_brands.append(model_counts)        

# Task A: Identify top 10 brands by frequency

In [127]:
top_brands

[('car', 2289),
 ('bmw', 1595),
 ('audi', 1259),
 ('acura', 1158),
 ('hyundai', 769),
 ('volkswagen', 730),
 ('honda', 675),
 ('infiniti', 427),
 ('ford', 403),
 ('kia', 390),
 ('chrysler', 374),
 ('seat', 297),
 ('sedan', 292),
 ('problem', 273),
 ('cadillac', 259),
 ('mercedesbenz', 255),
 ('subaru', 227),
 ('volvo', 182),
 ('toyota', 141),
 ('nissan', 138),
 ('chevrolet', 129)]

# Calculate lift ratios for associations between brands 